In [227]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

: 

In [228]:
df = pd.read_csv('data.csv')
df

,hour_of_day,day_of_week,is_holiday,season,temperature,humidity,wind_speed,rainfall,Peak_Demand_met_(MW),Load_Growth_(%),Energy_Consumption_(MUs),Shedding_(MUs),Shedding_as_of_Energy_Consumption,Transmission_Losses_(%),System_Availability_(%),electricity_load_MW
0,6,3,False,summer,44.260715,49.279758,8.979877,1.560186,6955.833307,32.484355,162.397808,2.904181,1.788313,4.464705,98.005575,6955.833307
1,21,4,False,summer,44.548648,53.297706,3.185087,1.818250,7010.409445,7.473748,173.361804,15.212112,8.774777,3.099026,97.159725,7010.409445
2,9,3,False,summer,44.606333,29.310854,1.359097,6.183860,6773.425175,30.818452,252.984797,49.161544,19.432608,2.867052,99.299702,6773.425175
3,8,6,False,summer,30.198974,57.688070,8.449323,3.854165,6836.829423,26.599799,106.386501,11.544691,10.851651,1.964102,98.416318,6836.829423
4,7,6,False,summer,32.600470,35.642424,2.733541,7.553614,6736.447733,40.537086,270.062350,10.397083,3.849882,3.270801,95.156566,6736.447733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,1,4,False,winter,5.317482,44.334988,3.975813,0.654637,2000.000000,-2.871025,341.052960,49.906506,14.633066,1.403261,99.106151,2000.000000
2496,1,3,True,winter,14.647364,76.108677,1.709184,3.154445,2000.000000,33.374006,136.311904,20.369166,14.943058,4.646373,98.907616,2000.000000
2497,18,2,False,winter,18.252286,57.977672,2.245285,8.375852,2000.000000,26.424918,211.855331,2.681677,1.265806,1.504152,98.045295,2000.000000
2498,14,0,False,winter,15.907702,70.945606,13.807539,5.849375,2000.000000,18.708944,411.769346,37.654984,9.144679,2.031236,96.551130,2000.000000


In [229]:
df.season.value_counts()

,count
season,
summer,625
rainy,625
autumn,625
winter,625


In [230]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [231]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25 , random_state = 1)

In [232]:
cat_cols = ['is_holiday','season']
num_cols = ['hour_of_day',
            'day_of_week','temperature','humidity',	'wind_speed',	'rainfall',	'Peak_Demand_met_(MW)',	'Load_Growth_(%)',	'Energy_Consumption_(MUs)',	'Shedding_(MUs)',	'Shedding_as_of_Energy_Consumption',	'Transmission_Losses_(%)','System_Availability_(%)']

In [233]:
# ML Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [234]:
num_pipeline = Pipeline(
    steps = [
              ('imputation',SimpleImputer(strategy = 'median')),
              ('scaling',StandardScaler())
            ]
)

cat_pipeline = Pipeline(
    steps = [
              ('imputation',SimpleImputer(strategy = 'most_frequent')),
              ('encoding',OneHotEncoder(handle_unknown='ignore'))
            ]
)

In [235]:
# columns transformer
col_tran = ColumnTransformer([
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
]
)
X_train = col_tran.fit_transform(X_train)
X_test = col_tran.transform(X_test)

In [236]:
# Accuracy of Different Regression Models
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

models = {
    'Decision Tree' : DecisionTreeRegressor(),
    'Support Vector Regressor' : SVR(),
    'Linear Regressor' : LinearRegression(),
    'Random Forest' : RandomForestRegressor()
}

In [237]:
from sklearn.metrics import r2_score

def model_evaluation(X_train,y_train,X_test,y_test,models):
  eval = {}

  for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)
    performance = r2_score(y_test,y_pred)
    eval[list(models.keys())[i]] = performance

  return eval

model_evaluation(X_train,y_train,X_test,y_test,models)

{'Decision Tree': 0.999947507497478,
 'Support Vector Regressor': 0.07819390947695848,
 'Linear Regressor': 1.0,
 'Random Forest': 0.9999872894821509}

In [238]:
rf = RandomForestRegressor()

from sklearn.model_selection import RandomizedSearchCV
params = {
            'max_depth' : [1,2,3,5,10,None],
            'n_estimators' : [50,100,200,300],
            'criterion' : ['squared_error', 'absolute_error']
}

reg = RandomizedSearchCV(rf,param_distributions=params,cv=5,verbose=3,scoring='neg_mean_squared_error')
reg.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=absolute_error, max_depth=10, n_estimators=50;, score=-307.273 total time=   6.8s
[CV 2/5] END criterion=absolute_error, max_depth=10, n_estimators=50;, score=-426.418 total time=   5.4s
[CV 3/5] END criterion=absolute_error, max_depth=10, n_estimators=50;, score=-38.062 total time=   6.8s
[CV 4/5] END criterion=absolute_error, max_depth=10, n_estimators=50;, score=-49.625 total time=   5.5s
[CV 5/5] END criterion=absolute_error, max_depth=10, n_estimators=50;, score=-36.222 total time=   6.2s
[CV 1/5] END criterion=absolute_error, max_depth=None, n_estimators=50;, score=-365.196 total time=   6.0s
[CV 2/5] END criterion=absolute_error, max_depth=None, n_estimators=50;, score=-300.643 total time=   5.7s
[CV 3/5] END criterion=absolute_error, max_depth=None, n_estimators=50;, score=-42.853 total time=   6.8s
[CV 4/5] END criterion=absolute_error, max_depth=None, n_estimators=50;, score=-70.161 total time

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error'],
                                        'max_depth': [1, 2, 3, 5, 10, None],
                                        'n_estimators': [50, 100, 200, 300]},
                   scoring='neg_mean_squared_error', verbose=3)

In [239]:
best_model = reg.best_estimator_
y_pred_tune = best_model.predict(X_test)
r2_score(y_test,y_pred_tune)

0.9999776940473272

In [241]:
# Model Saving
import pickle
filename = 'model_main.sav'
pickle.dump(best_model, open(filename, 'wb'))

In [242]:
import joblib
joblib.dump(col_tran, 'column_transformer_m.sav')

['column_transformer_m.sav']